In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from scipy import stats
import lightgbm as lgb
from autogluon.tabular import TabularDataset, TabularPredictor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# реализую два класса, чтобы каждый раз не запускать 100500 ячеек: 
# DATA_GETTER будет загружать данные и делать feature_engineering для создания экстра признаков
# после я буду добавлять признаки "аномальности", полученные либо с помощью методов поиска аномалий, либо с помощью кластеризации
# LightGBMpipe будет на новых данных обучать модель LightGBM с помощью кросс-валидации, оценивать ее качество

class DATA_GETTER:
    def __init__(self, ):
        INPUT_DIR = '.'
        train_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'train_transaction.csv'))
        train_identity = pd.read_csv(os.path.join(INPUT_DIR, 'train_identity.csv'))
        test_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'test_transaction.csv'))
        test_identity = pd.read_csv(os.path.join(INPUT_DIR, 'test_identity.csv'))
        self.sample_submission = pd.read_csv(os.path.join(INPUT_DIR, 'sample_submission.csv'))
        self.df_train = train_transaction.merge(train_identity, how='left', on='TransactionID')
        self.df_test = test_transaction.merge(test_identity, how='left', on='TransactionID')
        self.feature_engineering()

    def feature_engineering(self):
        # обработаем признаки с доменами
        for col in ['P_emaildomain', 'R_emaildomain']:
            self.df_train[col + '_suffix'] = self.df_train[col].map(lambda x: str(x).split('.')[-1])
            self.df_test[col + '_suffix'] = self.df_test[col].map(lambda x: str(x).split('.')[-1])

        self.df_train['same_emaildomain'] = (self.df_train['P_emaildomain'] == self.df_train['R_emaildomain']).astype('uint8')
        self.df_test['same_emaildomain'] = (self.df_test['P_emaildomain'] == self.df_test['R_emaildomain']).astype('uint8')

        # скомбинируем признаки, которые входят в топ-50 по важности признаков в бейзлайне
        pairs = [('card3', 'card5')]
        for col1, col2 in pairs:
            new_col_name = col1 + '_' + col2
            self.df_train[new_col_name] = self.df_train[col1].astype(str) + '_' + self.df_train[col2].astype(str)
            self.df_test[new_col_name] = self.df_test[col1].astype(str) + '_' + self.df_test[col2].astype(str)

        # мета-признаки - исходя из частоты встречаемости в выборке
        for col in ['card1', 'card2']:#, 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'DeviceType', 'DeviceInfo']:
            card_freq = self.df_train[col].value_counts().to_dict()
            self.df_train['{}_cnt'.format(col)] = self.df_train[col].map(card_freq)
            self.df_test['{}_cnt'.format(col)] = self.df_test[col].map(card_freq)

        # используем агрегированние
        for col in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'DeviceType', 'DeviceInfo']:
            for agg_type in ['mean', 'median', 'min', 'max']:
                agg_col_name = 'TransactionAmt_{}_{}'.format(col, agg_type)
                card_agg = self.df_train.groupby(col)['TransactionAmt'].agg([agg_type]).rename({agg_type: agg_col_name}, axis=1)
                self.df_train = self.df_train.merge(card_agg, how='left', on=col)
                self.df_test = self.df_test.merge(card_agg, how='left', on=col)
       

        # прологарифируем смещенный признак TransactionAmt
        self.df_train['TransactionAmt_log'] = np.log1p(self.df_train['TransactionAmt'])
        self.df_test['TransactionAmt_log'] = np.log1p(self.df_test['TransactionAmt'])

        # достанем признак "Центы"
        self.df_train['TransactionAmt_Cents'] = np.modf(self.df_train['TransactionAmt'])[0] * 100
        self.df_test['TransactionAmt_Cents'] = np.modf(self.df_test['TransactionAmt'])[0] * 100

        # удалим изначальный признак
        self.df_train.drop('TransactionAmt', axis=1, inplace=True)
        self.df_test.drop('TransactionAmt', axis=1, inplace=True)

        # заполним пропуски в категориальных признаках значением "unseen_category", а в вещественных - "-1"
        for col in tqdm(self.df_train.columns.drop('isFraud')):
            if self.df_train[col].dtype == 'O':
                self.df_train[col] = self.df_train[col].fillna('unseen_category')
                self.df_test[col] = self.df_test[col].fillna('unseen_category')
                self.df_train[col] = self.df_train[col].astype('category')
                self.df_test[col] = self.df_test[col].astype('category')
            else:
                median = np.median(self.df_train[col].dropna())
                self.df_train[col] = self.df_train[col].fillna(median)
                self.df_test[col] = self.df_test[col].fillna(median)
                # self.df_train[col] = self.df_train[col].fillna(-1)
                # self.df_test[col] = self.df_test[col].fillna(-1)


    
class LightGBMpipe:
    def __init__(self, df_train, df_test, model_params=None):
        params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'metric': 'auc',
            'n_jobs': -1,
            'n_estimators': 2000,
            'seed': 13,
            'early_stopping_rounds': 200,
            'verbosity': -1
        }
        if model_params is None:
            self.params = params
        else:
            self.params = model_params

        self.scores = []
        self.feature_importances = pd.DataFrame()
        self.feature_importances['feature'] = df_train.drop(columns=['isFraud', 'TransactionID', 'TransactionDT']).columns
        self.test_preds = []

        # выделяем фолды
        month_length = 3600 * 24 * 30
        fold0_idx = df_train[df_train['TransactionDT'] < df_train['TransactionDT'].min() + month_length].index
        fold1_idx = df_train[(df_train['TransactionDT'].min() + month_length <= df_train['TransactionDT']) & (df_train['TransactionDT'] < df_train['TransactionDT'].min() + 2 * month_length)].index
        fold2_idx = df_train[(df_train['TransactionDT'].min() + 2 * month_length <= df_train['TransactionDT']) & (df_train['TransactionDT'] < df_train['TransactionDT'].min() + 3 * month_length)].index
        fold3_idx = df_train[df_train['TransactionDT'].min() + 3 * month_length <= df_train['TransactionDT']].index
        self.folds_idx = [fold0_idx, fold1_idx, fold2_idx, fold3_idx]

        # выделяем идентификационный и временной признаки и удаляем полные дубликаты
        self.df_train = df_train.drop(columns=['TransactionID', 'TransactionDT'])
        self.df_test = df_test.drop(columns=['TransactionID', 'TransactionDT'])

    def fit(self):
        for i in tqdm(range(len(self.folds_idx))):
            X_train = self.df_train.drop(index=self.folds_idx[i], errors='ignore')
            y_train = X_train['isFraud'].values
            X_val = self.df_train.iloc[self.folds_idx[i]]
            y_val = X_val['isFraud'].values
            X_train = X_train.drop('isFraud', axis=1)
            X_val = X_val.drop('isFraud', axis=1)

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
            lgb_model = lgb.train(self.params, lgb_train, valid_sets=lgb_eval)

            self.feature_importances['fold_{}'.format(i)] = lgb_model.feature_importance()

            y_pred = lgb_model.predict(X_val)
            score_fold = roc_auc_score(y_val, y_pred)
            self.scores.append(score_fold)
            y_test_pred = lgb_model.predict(self.df_test)
            self.test_preds.append(y_test_pred)

        for i in range(len(self.scores)):
            print('Fold {}, AUC-ROC: {:.5f}'.format(i, self.scores[i]))
        print('CV AUC-ROC: {:.5f}'.format(np.mean(self.scores)))
        
        fold_cols = [col for col in self.feature_importances.columns if col.startswith('fold_')]
        self.feature_importances['average'] = self.feature_importances[fold_cols].mean(axis=1)
        return np.mean(self.scores)
    
    def plot_feature_importances(self):    
        plt.figure(figsize=(15, 15))
        sns.barplot(data=self.feature_importances.sort_values(by='average', ascending=False).head(50), x='average', y='feature', palette="BuGn_r")
        plt.title('Top feature importances')
        plt.show()

    def save_predict(self, name='test_try'):
        # сохраняем сабмишн
        final_pred = np.average(self.test_preds, axis=0)
        sub = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': final_pred})
        sub.to_csv('submissions/' + name + '.csv', index=False)

In [3]:
data_getter = DATA_GETTER()
df_train, df_test, sample_submission = data_getter.df_train, data_getter.df_test, data_getter.sample_submission

100%|██████████| 480/480 [00:02<00:00, 177.00it/s]


In [19]:
# df_train['V_sum'] = 0
# df_test['V_sum'] = 0
# for col in df_train.columns[df_train.columns.str.startswith('V')].to_list():
#     df_train['V_sum'] += df_train[col]
#     df_test['V_sum'] += df_test[col]

In [ ]:
# boxplot
# num_cols = ['TransactionAmt_log', 'card1', 'card2', 'card3', 'card5', 'dist1', 'dist2']
# df_train['sum_is_outlier'] = 0
# df_test['sum_is_outlier'] = 0
# for col in tqdm(num_cols):
#     Q1 = df_train[col].quantile(0.25)
#     Q3 = df_train[col].quantile(0.75)
#     IQR = Q3 - Q1
#     df_train[col+'_is_outlier'] = np.where(((df_train[col] < (Q1 - 1.5 * IQR)) | (df_train[col] > (Q3 + 1.5 * IQR))), 1, 0)
#     df_test[col+'_is_outlier'] = np.where(((df_test[col] < (Q1 - 1.5 * IQR)) | (df_test[col] > (Q3 + 1.5 * IQR))), 1, 0)
    
#     df_train['sum_is_outlier'] += df_train[col+'_is_outlier']
#     df_test['sum_is_outlier'] += df_test[col+'_is_outlier']

# drop_cols = [col+'_is_outlier' for col in num_cols]
# df_train = df_train.drop(columns=drop_cols)
# df_test = df_test.drop(columns=drop_cols)

100%|██████████| 7/7 [00:00<00:00, 92.52it/s]


In [47]:
pipe = LightGBMpipe(df_train=df_train, df_test=df_test)
pipe.fit()

100%|██████████| 4/4 [00:55<00:00, 13.89s/it]

Fold 0, AUC-ROC: 0.90806
Fold 1, AUC-ROC: 0.93027
Fold 2, AUC-ROC: 0.93335
Fold 3, AUC-ROC: 0.91207
CV AUC-ROC: 0.92094


0.9209357484181417

In [40]:
pipe.save_predict('default2')

In [233]:
# X, y = df_train.drop(columns=['isFraud']), df_train['isFraud']
# num_cols = [col for col in X.columns if X[col].dtype in ('int64', 'float64')]
# cat_cols = [col for col in X.columns if X[col].dtype in ('O', 'category')]
# ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# cat_data = pd.DataFrame(ohe.fit_transform(X[cat_cols]), columns=ohe.get_feature_names_out()).reset_index(drop=True)
# scaler = StandardScaler()
# num_data = pd.DataFrame(scaler.fit_transform(X[num_cols]), columns=num_cols).reset_index(drop=True)
# X = pd.concat([cat_data, num_data], axis=1)

In [234]:
# num_data_test = pd.DataFrame(scaler.transform(df_test[num_cols]), columns=num_cols).reset_index(drop=True)
# cat_data_test = pd.DataFrame(ohe.transform(df_test[cat_cols]), columns=ohe.get_feature_names_out()).reset_index(drop=True)
# X_test = pd.concat([cat_data_test, num_data_test], axis=1)

In [ ]:
# pipe.save_predict(name='median filling nan')  # 0.92142
# pipe.save_predict(name='M_SUM indicator')  # 0.91974
# pipe.save_predict(name='2200 iters')  # 0.9
# pipe.save_predict(name='MORE_boxplot_features')  # 0.91890
# pipe.save_predict(name='only_zeros')  # 0.5
# pipe.save_predict(name='original lightgbm')  # 0.92142
# pipe.save_predict(name='top100_features')  # 0.9

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

pipeline = make_pipeline(
    RandomUnderSampler(),  # Сначала применяем андерсэмплинг
    SMOTE()                # Затем применяем SMOTE
)

X_resampled, y_resampled = pipeline.fit_resample(X, y)

In [49]:
def objective_lightgbm(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.3, log=True)
    feature_fraction = trial.suggest_float("feature_fraction", 0.4, 1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 800, 4000)
    max_depth = trial.suggest_int("max_depth", 3, 17)
    lambda_l1 = trial.suggest_float('lambda_l1', 0, 100)
    lambda_l2 = trial.suggest_float('lambda_l2', 0, 100)
    num_boost_round = trial.suggest_int('num_boost_round', 0, 100)

    params = {
        'objective': 'binary',
            'boosting_type': 'gbdt',
            'metric': 'auc',
            'n_jobs': -1,
            'learning_rate': learning_rate,
            'n_estimators': n_estimators,
            'seed': 13,
            'early_stopping_rounds': 200,
            'verbosity': -1,
            'max_depth': max_depth,
            'feature_fraction': feature_fraction,
            'lambda_l1': lambda_l1,
            'lambda_l2': lambda_l2,
            'num_boost_round': num_boost_round
    }

    pipe = LightGBMpipe(df_train=df_train, df_test=df_test, model_params=params)
    roc_auc = pipe.fit()
    return roc_auc

In [52]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective_lightgbm, n_trials=500)
study.best_params, study.best_value

[I 2025-03-12 19:38:16,450] A new study created in memory with name: no-name-d86fdce2-5bc3-4ace-a131-f0774b3ac81a
100%|██████████| 4/4 [00:54<00:00, 13.71s/it]
[I 2025-03-12 19:39:12,447] Trial 0 finished with value: 0.9035454489210216 and parameters: {'learning_rate': 0.044298005520585104, 'feature_fraction': 0.849859453506555, 'n_estimators': 3165, 'max_depth': 12, 'lambda_l1': 53.08590676844063, 'lambda_l2': 37.74387489151353, 'num_boost_round': 50}. Best is trial 0 with value: 0.9035454489210216.


Fold 0, AUC-ROC: 0.88752
Fold 1, AUC-ROC: 0.91138
Fold 2, AUC-ROC: 0.91590
Fold 3, AUC-ROC: 0.89938
CV AUC-ROC: 0.90355


100%|██████████| 4/4 [00:47<00:00, 11.88s/it]
[I 2025-03-12 19:40:00,928] Trial 1 finished with value: 0.8998220293801467 and parameters: {'learning_rate': 0.06248154125047334, 'feature_fraction': 0.6225542544961791, 'n_estimators': 2612, 'max_depth': 10, 'lambda_l1': 65.31640422619253, 'lambda_l2': 69.58588678272558, 'num_boost_round': 73}. Best is trial 0 with value: 0.9035454489210216.


Fold 0, AUC-ROC: 0.88324
Fold 1, AUC-ROC: 0.90664
Fold 2, AUC-ROC: 0.91202
Fold 3, AUC-ROC: 0.89738
CV AUC-ROC: 0.89982


100%|██████████| 4/4 [01:54<00:00, 28.54s/it]
[I 2025-03-12 19:41:56,140] Trial 2 finished with value: 0.8970901362885227 and parameters: {'learning_rate': 0.01604875350300337, 'feature_fraction': 0.5467262603110135, 'n_estimators': 3648, 'max_depth': 9, 'lambda_l1': 82.36967898545723, 'lambda_l2': 10.209527718154842, 'num_boost_round': 37}. Best is trial 0 with value: 0.9035454489210216.


...

100%|██████████| 4/4 [03:05<00:00, 46.42s/it]
[I 2025-03-14 04:49:51,238] Trial 498 finished with value: 0.9222434379885818 and parameters: {'learning_rate': 0.037753268937009296, 'feature_fraction': 0.4091208217620358, 'n_estimators': 1913, 'max_depth': 14, 'lambda_l1': 6.005236547044152, 'lambda_l2': 1.8001292346747897, 'num_boost_round': 45}. Best is trial 452 with value: 0.9256627859438993.


Fold 0, AUC-ROC: 0.90794
Fold 1, AUC-ROC: 0.93381
Fold 2, AUC-ROC: 0.93411
Fold 3, AUC-ROC: 0.91311
CV AUC-ROC: 0.92224


100%|██████████| 4/4 [03:20<00:00, 50.03s/it]
[I 2025-03-14 04:53:12,369] Trial 499 finished with value: 0.9225167484732583 and parameters: {'learning_rate': 0.03012282659966281, 'feature_fraction': 0.4061660575523601, 'n_estimators': 1878, 'max_depth': 13, 'lambda_l1': 3.5573191514829827, 'lambda_l2': 3.844250259062087, 'num_boost_round': 38}. Best is trial 452 with value: 0.9256627859438993.


Fold 0, AUC-ROC: 0.90911
Fold 1, AUC-ROC: 0.93448
Fold 2, AUC-ROC: 0.93458
Fold 3, AUC-ROC: 0.91190
CV AUC-ROC: 0.92252


({'learning_rate': 0.03073775816915715,
  'feature_fraction': 0.4113379114637828,
  'n_estimators': 2168,
  'max_depth': 13,
  'lambda_l1': 1.7393414529389066,
  'lambda_l2': 0.02428271022480627,
  'num_boost_round': 45},
 0.9256627859438993)

In [53]:
study.best_params, study.best_value

({'learning_rate': 0.03073775816915715,
  'feature_fraction': 0.4113379114637828,
  'n_estimators': 2168,
  'max_depth': 13,
  'lambda_l1': 1.7393414529389066,
  'lambda_l2': 0.02428271022480627,
  'num_boost_round': 45},
 0.9256627859438993)

In [ ]:
best_params = {'learning_rate': 0.03073775816915715,
  'feature_fraction': 0.4113379114637828,
  'n_estimators': 2168,
  'max_depth': 13,
  'lambda_l1': 1.7393414529389066,
  'lambda_l2': 0.02428271022480627,
  'num_boost_round': 45}



pipe = LightGBMpipe(df_train=df_train, df_test=df_test, model_params=best_params)
pipe.fit()
pipe.save_predict('best_params_lightGBM_33hours')

100%|██████████| 4/4 [03:13<00:00, 48.40s/it]

Fold 0, AUC-ROC: 0.89485
Fold 1, AUC-ROC: 0.92254
Fold 2, AUC-ROC: 0.92355
Fold 3, AUC-ROC: 0.89915
CV AUC-ROC: 0.91002


In [55]:
best_params2 = {'learning_rate': 0.028302997383156236, 'feature_fraction': 0.41520061626400007, 'n_estimators': 1910, 'max_depth': 14, 'lambda_l1': 3.965000680369279, 'lambda_l2': 0.05169416536383464, 'num_boost_round': 43}
pipe = LightGBMpipe(df_train=df_train, df_test=df_test, model_params=best_params2)
pipe.fit()
pipe.save_predict('best_params_lightGBM_33hours_2')

100%|██████████| 4/4 [02:55<00:00, 43.91s/it]

Fold 0, AUC-ROC: 0.89277
Fold 1, AUC-ROC: 0.92144
Fold 2, AUC-ROC: 0.92317
Fold 3, AUC-ROC: 0.90048
CV AUC-ROC: 0.90946


In [6]:
# baseline: 0.92094
params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'metric': 'auc',
            'n_jobs': -1,
            'n_estimators': 2200,
            'seed': 13,
            'early_stopping_rounds': 200,
            'verbosity': -1,
            # 'num_boost_round': 43
        }
# pipe = LightGBMpipe(df_train=df_train, df_test=df_test, model_params=params)
# pipe.fit()
pipe.save_predict('default__')